# Chatbot with OpenAI

In this notebook, we are going to implement, step-by-step, a chatbot using the OpenAI API.

Let's start by installing the required libraries.

In [3]:
!pip install openai
!pip install gradio
!pip install ipython

In [4]:
from IPython.display import Markdown, display


# Helper function to display markdown content in the notebook
def printmd(string):
    display(Markdown(string))

## Load the environment variables

This step is similiar to what we implemented in the previous notebook. We will load the environment variables from the `.env` file.

In [5]:
from dotenv import load_dotenv

In [6]:
_ = load_dotenv('.env')

Now, let's load the OpenAI library, and set our API key.

In [7]:
import openai
import os

In [8]:
openai.api_key = os.getenv('OPENAI_API_KEY')

## Let's change the completion function

We will use a different completion function, but it is very similar to the one we used in the previous notebook. We will then create a function, based on ths one, that is suitable for chatbots.

In [9]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0):
    """
    Get the completion from OpenAI API.
    :param messages: the messages to be sent to the chatbot
    :param model: the model to be used
    :param temperature: the temperature of the completion
    :return: the completion
    """
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    completion = response.choices[0].message["content"]
    return completion

## Roles

In OpenAI API, we can define the roles of the agents in the conversation. 

In our case, we will have three roles: `system`, `user`, and `assistant`. The `system` role is used to set the behaviour of the chatbot (i.e., the  personality). The `user` role is the user that is interacting with the chatbot.  The `assistant` role is the chatbot itself, i.e., the chat model.

In ChatGPT web interface, your messages are the `user` messages, and then ChatGPT's are the `assistant` messages. 

In [10]:
messages = [
    {"role": "system", "content": "You are a teacher..."},
    {"role": "user",
     "content": "Explain how an artificial neural network works. Don't use more than 50 words."}
]

printmd(get_completion(messages))

An artificial neural network is a computer system that learns and makes decisions by mimicking the structure and function of the human brain. It consists of interconnected nodes, or "neurons," that process and transmit information to solve complex problems.

_**Do it your self:**_ Try to change the `system` message to something else, and see how the chatbot behaves.

In [11]:
messages = [
    {"role": "system", "content": "You are a children..."},
    {"role": "user",
     "content": "Explain how an artificial neural network works. Don't use more than 50 words."}
]

printmd(get_completion(messages))

An artificial neural network is like a brain made by computers. It has lots of tiny parts called neurons that work together to solve problems. It learns by getting examples and adjusting its connections. It's really smart!

_**Do it your self:**_ Now, stop here and think how can we continue the conversation. I.e., how can we make the chatbot continuing the conversation? How can the chatbot understand the context of the conversation.?

In [12]:
messages = [
    {"role": "system", "content": "You are a children..."},
    {"role": "user",
     "content": "Explain how an artificial neural network works. Don't use more than 50 words."},
    {"role": "assistant",
     "content": "An artificial neural network is like a brain made by computers. It has lots of tiny parts called neurons that work together to solve problems. It learns by practicing and adjusting its connections until it gets better at what it does."},
    {"role": "user",
     "content": "Can you give me more information about the functions of the neurons."}
]

printmd(get_completion(messages))

Sure! Neurons in an artificial neural network are like little decision-makers. They take in information from other neurons, process it, and then decide whether to send a signal to the next neuron or not. They do this by using mathematical functions that help them make decisions based on the input they receive. These functions can be simple or complex, depending on the task the neural network is trying to solve.

At this point, we can see what we need to do to make the chatbot continuing the conversation. We need to keep track of the conversation, and then use the last message to generate the next one. I.e., we need to keep track of the context of the conversation by storing the messages from the `user` and the `assistant`.

In [13]:
context = [
    {"role": "system", "content": "You are an assitant..."}
]

In [14]:
def chat(prompt, history=None):
    context.append({"role": "assistant", "content": prompt})
    completion = get_completion(context)
    context.append(
        {"role": "assistant", "content": completion})
    return completion

In [15]:
printmd(chat(
    "Explain how an artificial neural network works. Don't use more than 50 words."))

An artificial neural network is a computational model inspired by the structure and function of the human brain. It consists of interconnected nodes, or "neurons," that process and transmit information. Through a process of learning and adjustment, the network can recognize patterns and make predictions.

In [16]:
printmd(chat("Hum... it's still not very clear."))


An artificial neural network is a computational model that mimics the structure and function of the human brain. It consists of interconnected nodes, or "neurons," that process and transmit information. By learning from data, the network can recognize patterns and make predictions.

In [17]:
printmd(chat(
    "Can you give me more information about the functions of the neurons."))

Certainly! In an artificial neural network, each neuron performs a simple mathematical operation on the input it receives. This operation typically involves multiplying the input by a weight, summing up the weighted inputs, and applying an activation function to determine the output of the neuron. The activation function introduces non-linearity to the network, allowing it to learn complex patterns and make more accurate predictions.

## Real-world example

In this example, we are going to read a text file with the information about the AI summit, and then use the chatbot to answer questions about the text.

In [18]:
with open('info.md', 'r') as file:
    event_info = file.read()

In [19]:
context = [
    {"role": "system",
     "content": "You are an organizer of the event, and you only reply to questions related to the event. Here is everything you need to know about the event: ```" + event_info + "```"}
]

In [20]:
printmd(chat("What is the name of the event?"))

The name of the event is the AI Summit.

In [21]:
printmd(chat("What is the date of the event?"))

The AI Summit will take place on September 13, 2023.

In [22]:
printmd(chat("What are the names of the speakers?"))

The names of the speakers are:

- João Santos from Microsoft
- David Jardim from Oracle
- Eliano Marques from Tracer
- Adi Mazor Kario from Invincible Innovation

In [23]:
printmd(chat("What is the name of the venue?"))

The event will take place at Museu da Imprensa, the Press Museum.

In [24]:
printmd(chat("Is the event free?"))

Yes, registration for the event is free.

In [25]:
printmd(chat("Is it suitable for beginners in AI?"))

Yes, the AI Summit is open to everyone, including beginners in AI. It is designed for professionals, researchers, students, entrepreneurs, technology fans, industry experts, government officials, and anyone else interested in Artificial Intelligence.

In [26]:
printmd(chat("When is the coffee break?"))

The coffee break will take place at 16:00 during the event.

## Chatbot web app

Using the gradio library, we can create a simple web app for our chatbot.

In [27]:
import gradio as gr

In [28]:
gr.ChatInterface(
    chat,
    chatbot=gr.Chatbot(),
    title="Welcome to the AI Summit chatbot",
    description="Ask me a question about the event...",
    theme="monochrome",
    examples=['how much does it cost?', 'what is the date of the event?',
              'what is the name of the venue?']
).launch(share=True)


Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://085a752ba0154324fd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
